Final Project w207

Matthew Holmes
Andrew Kabatznick
Grant Whittington

Data from: https://www.kaggle.com/c/bike-sharing-demand/data

In [25]:
%matplotlib inline

##basically just importanting most of the libraries from previous assignments

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from matplotlib.colors import LogNorm
from sklearn import tree

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

In [4]:
def transform_raw_data(raw_input):
    #Putting Data into Pandas dataframe
        Column_Headers = csvfile.readline().split(',')
        Column_Headers[-1] = "totalcount"
        csvfile.next()
        train_data = []
        for row in csvfile:
            new_row = row.replace('\n', '')
            train_data.append(new_row.split(','))
        
        df = pd.DataFrame(data = train_data[1:-1], columns = Column_Headers)
        return df

In [5]:
with open('train.csv') as csvfile:
    train_data = transform_raw_data(csvfile)
with open('test.csv') as csvfile:
    test_data = transform_raw_data(csvfile)
a = train_data['datetime']
[a,b] = a.iloc[1].split()
[c,d,e]=a.split('-')

In [6]:
##Need to do train/dev split, also modify/add any features we desire
def modify_factors(input_data):
    output_data=pd.DataFrame(input_data)
    ##Split datetime into year, month, day, and hour columns
    rawdate = input_data['datetime']
    for i in range(len(rawdate)):
        [date,hour] = rawdate.iloc[i].split()
        [year,month,day] = date.split("-")
        output_data['year'] = int(year)
        output_data['month']=int(month)
        output_data['day']=int(day)
        output_data['hour']=int(hour[:2])
    ##One hot encoding for season, weather
    season_onehot = pd.get_dummies(input_data['season'])
    season_onehot.columns = ["winter", "spring", "summer", "autumn"]
    output_data = pd.concat([output_data,season_onehot],axis=1)
    
    weather_onehot = pd.get_dummies(input_data['weather'])
    weather_onehot.columns = ["clear", "misty", "light_precip", "heavy_precip"]
    output_data = pd.concat([output_data,weather_onehot],axis=1)
    
    del output_data['datetime']
    del output_data['season']
    del output_data['weather']
    
    return output_data

train_data = modify_factors(train_data)

In [7]:
#Splitting Columns into Prediction and Numerical vs Categorical
All_Columns = list(train_data.columns.values)
Prediction_Columns = All_Columns[:]
Prediction_Columns.remove('registered')
Prediction_Columns.remove('casual')
Prediction_Columns.remove('totalcount')
Numerical_Predictors = Prediction_Columns[0:9 or None]
Categorical_Predictors = Prediction_Columns[10:-0 or None]

In [28]:
#train, dev, test split of base data
def data_split(data,train_percent,dev_percent,is_random):
    size_of_data=len(data)
    train_amount=int(round(train_percent*size_of_data))
    dev_amount=int(round(dev_percent*size_of_data))
    train, dev, test = np.split(data.sample(frac=1), [train_amount, train_amount+dev_amount])
    return train,dev,test
[train_set,dev_set,test_set]=data_split(train_data,.45,.3,1)

In [33]:
#Basic Linear Regression using numerical predictors
LinearReg = LinearRegression()
LinearReg.fit(train_set[Prediction_Columns], train_set['totalcount'])
print LinearReg.score(dev_set[Prediction_Columns], dev_set['totalcount'])

0.274572154876


In [ ]:
##Investigate using bins for totalcount to improve below methods

In [36]:
#Multinomial Naive Bayes
MultiBayes = MultinomialNB()
MultiBayes.fit(train_set[Prediction_Columns], train_set['totalcount'])
print MultiBayes.score(dev_set[Prediction_Columns], dev_set['totalcount'])

0.00857580398162


In [40]:
#Decision Trees
DTree = tree.DecisionTreeClassifier()
DTree.fit(train_set[Prediction_Columns], train_set['totalcount'])
print DTree.score(dev_set[Prediction_Columns], dev_set['totalcount'])

0.00704441041348


In [ ]:
#Need to investigate a better way to score these things

In [ ]:
#Pruning?  Worth doing at all?